In [11]:
import pandas as pd
from datetime import datetime
import numpy as np
#INPUT = 'CompleteIndexes.csv'
#OUTPUT = 'MovingAverageWeeklyImputed.csv'
INPUT = 'CompleteWeeklyIndexes.csv'
OUTPUT = 'MovingAverageWeeklyImputed.csv'

In [12]:
df = pd.read_csv(INPUT, parse_dates=True, index_col=0)
df = df.loc['2000-01-01':'2015-01-01'] #Selecting training data
df.head(30)

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-07,1441.469971,1.225200e+09,21.719999,0.0,11522.559570,184900000.0,3529.600098,1.634930e+09,18193.410156,0.0,...,15405.629883,0.0,943.880005,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-14,1465.150024,1.085900e+09,19.660000,0.0,11722.980469,266830000.0,3704.739990,1.656630e+09,18956.550781,0.0,...,15542.230469,0.0,973.859985,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-21,1441.359985,1.209800e+09,20.820000,0.0,11251.709961,205840000.0,3849.959961,1.923680e+09,18878.089844,0.0,...,15108.410156,0.0,951.510010,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-28,1360.160034,1.095800e+09,26.139999,0.0,10738.870117,183090000.0,3446.129883,1.616370e+09,19434.779297,0.0,...,16185.940430,0.0,948.210022,0.0,24.412899,0.00,NaN,71.914247,117.577146,0.033573
2000-02-04,1424.369995,1.045100e+09,21.540001,0.0,10963.799805,166590000.0,3874.370117,1.751450e+09,19763.130859,0.0,...,NaN,NaN,1010.429993,0.0,NaN,0.75,NaN,NaN,NaN,NaN
2000-02-11,1387.119995,1.025700e+09,24.420000,0.0,10425.209961,193540000.0,3968.889893,1.738590e+09,NaN,NaN,...,17380.300781,0.0,1014.169983,0.0,NaN,0.74,NaN,NaN,NaN,NaN
2000-02-18,1346.089966,1.042300e+09,26.000000,0.0,10219.519531,208930000.0,3965.750000,1.898410e+09,19789.029297,0.0,...,16599.160156,0.0,992.530029,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-02-25,1333.359985,1.065200e+09,25.200001,0.0,9862.120117,195240000.0,4178.580078,1.825500e+09,19817.880859,0.0,...,17200.980469,0.0,1004.719971,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-03-03,1409.170044,1.150300e+09,19.209999,0.0,10367.200195,210460000.0,4442.870117,2.136530e+09,19927.539063,0.0,...,17285.240234,0.0,1042.400024,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-03-10,1395.069946,1.138800e+09,21.240000,0.0,9928.820313,197440000.0,4587.160156,1.992170e+09,19750.400391,0.0,...,17831.859375,0.0,1045.270020,0.0,NaN,0.00,NaN,NaN,NaN,NaN


In [13]:
def imput_moving_average_to_col(df_to_input, num_values):
    for index, row in df_to_input.iterrows():
        if pd.isna(row[0]):
            #print("_______________", row)
            count_prev = 0
            mean = 0
            index_prev = df_to_input.index.get_loc(index)
            while count_prev < num_values:
                index_prev -= 1
                if index_prev < 0:
                    break
                if not pd.isna(df_to_input.iloc[index_prev][0]):
                    count_prev += 1
                    #print(df_to_input.iloc[index_prev])
                    mean += df_to_input.iloc[index_prev]
            count_next = 0
            index_next = df_to_input.index.get_loc(index)
            while count_next < num_values:
                index_next += 1
                if index_next >= len(df_to_input):
                    break
                if not pd.isna(df_to_input.iloc[index_next][0]):
                    count_next += 1
                    #print(df_to_input.iloc[index_next])
                    mean += df_to_input.iloc[index_next]
            mean = mean / (count_prev+count_next)
            #print("MEAN" , mean, "----------------")
            df_to_input.at[index, df_to_input.columns[0]]  = mean
    return df_to_input

In [9]:
for col in df.columns:
    df_imp = imput_moving_average_to_col(df[[col]],3)
    df[col] = df_imp
    print("Column", col, "was imputed.")
df.head()

Column sp500_Close was imputed.
Column sp500_Volume was imputed.
Column vix_Close was imputed.
Column vix_Volume was imputed.
Column dji_Close was imputed.
Column dji_Volume was imputed.
Column ndx_Close was imputed.
Column ndx_Volume was imputed.
Column n255_Close was imputed.
Column n255_Volume was imputed.
Column ftse_Close was imputed.
Column ftse_Volume was imputed.
Column hsi_Close was imputed.
Column hsi_Volume was imputed.
Column n100_Close was imputed.
Column n100_Volume was imputed.
Column Overall EMV Tracker was imputed.
Column infectious_daily_infect_emv_index was imputed.
Column GPR was imputed.
Column trade_US Trade Policy Uncertainty was imputed.
Column trade_Japanese Trade Policy Uncertainty was imputed.
Column trade_Trade Policy EMV Fraction was imputed.


,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-07,1441.469971,1.225200e+09,21.719999,0.0,11522.559570,184900000.0,3529.600098,1.634930e+09,18193.410156,0.0,...,15405.629883,0.0,943.880005,0.0,29.389908,0.00,38.115186,71.753826,84.038128,0.033312
2000-01-14,1465.150024,1.085900e+09,19.660000,0.0,11722.980469,266830000.0,3704.739990,1.656630e+09,18956.550781,0.0,...,15542.230469,0.0,973.859985,0.0,29.389908,0.00,38.115186,71.753826,84.038128,0.033312
2000-01-21,1441.359985,1.209800e+09,20.820000,0.0,11251.709961,205840000.0,3849.959961,1.923680e+09,18878.089844,0.0,...,15108.410156,0.0,951.510010,0.0,29.389908,0.00,38.115186,71.753826,84.038128,0.033312
2000-01-28,1360.160034,1.095800e+09,26.139999,0.0,10738.870117,183090000.0,3446.129883,1.616370e+09,19434.779297,0.0,...,16185.940430,0.0,948.210022,0.0,24.412899,0.00,38.115186,71.914247,117.577146,0.033573
2000-02-04,1424.369995,1.045100e+09,21.540001,0.0,10963.799805,166590000.0,3874.370117,1.751450e+09,19763.130859,0.0,...,16336.170410,0.0,1010.429993,0.0,31.563742,0.75,38.115186,84.473261,83.365366,0.033867


In [14]:
df.to_csv(OUTPUT, index = True)